In [1]:
import pandas as pd

from sklearn import datasets, model_selection, linear_model, neighbors

from evidently.dashboard import Dashboard
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.dashboard.tabs import ProbClassificationPerformanceTab

from evidently.model_profile import Profile
from evidently.model_profile.sections import ProbClassificationPerformanceProfileSection

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Iris Data

In [3]:
iris = datasets.load_iris()

In [4]:
iris_frame = pd.DataFrame(iris.data, columns = iris.feature_names)

In [5]:
iris_frame['target'] = iris.target

In [6]:
train_data, test_data = model_selection.train_test_split(iris_frame, random_state=0)

In [7]:
target = 'target'
prediction = 'prediction'

numerical_features = iris.feature_names
categorical_features = []

features = numerical_features + categorical_features

# Probabilistic Model Performance Dashboard 

In [8]:
model = linear_model.LogisticRegression()

In [9]:
model.fit(train_data[features], train_data.target)

LogisticRegression()

In [10]:
# make a data frame with predictions for each class

train_probas = pd.DataFrame(model.predict_proba(train_data[features]))
train_probas.columns = iris.target_names
test_probas = pd.DataFrame(model.predict_proba(test_data[features]))
test_probas.columns = iris.target_names

In [11]:
train_probas.head()

,setosa,versicolor,virginica
0,0.017768,0.860581,1.216512e-01
1,0.022194,0.934421,4.338548e-02
2,0.000036,0.073558,9.264061e-01
3,0.981263,0.018737,7.458927e-08
4,0.000152,0.139633,8.602151e-01


In [12]:
# get labels for target: [0, 1, 0, 2] -> ['setosa', 'versicolor', 'setosa', 'virginica']

train_data['target'] = [iris.target_names[x] for x in train_data['target']]
test_data['target'] = [iris.target_names[x] for x in test_data['target']]

In [13]:
# merge train and test data with predictions

train_data.reset_index(inplace=True, drop=True)
test_data.reset_index(inplace=True, drop=True)

merged_train_data = pd.concat([train_data, train_probas], axis=1)
merged_test_data = pd.concat([test_data, test_probas], axis=1)

In [14]:
merged_train_data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,setosa,versicolor,virginica
0,5.9,3.0,4.2,1.5,versicolor,0.017768,0.860581,1.216512e-01
1,5.8,2.6,4.0,1.2,versicolor,0.022194,0.934421,4.338548e-02
2,6.8,3.0,5.5,2.1,virginica,0.000036,0.073558,9.264061e-01
3,4.7,3.2,1.3,0.2,setosa,0.981263,0.018737,7.458927e-08
4,6.9,3.1,5.1,2.3,virginica,0.000152,0.139633,8.602151e-01


In [15]:
iris_column_mapping = ColumnMapping()

iris_column_mapping.target = 'target'
iris_column_mapping.prediction = iris.target_names.tolist()
iris_column_mapping.numerical_features = iris.feature_names

## Probabilistic Model Performance Dashboard full (verbose_level=1)

In [16]:
iris_prob_classification_dashboard = Dashboard(tabs=[ProbClassificationPerformanceTab(verbose_level=1)])
iris_prob_classification_dashboard.calculate(merged_train_data, merged_test_data, 
                                             column_mapping = iris_column_mapping)
iris_prob_classification_dashboard.show()

In [18]:
from evidently.report import Report
from evidently.metrics import ClassificationPerformanceMetrics

# bcancer_column_mapping.pos_label = 'malignant'
report = Report(metrics=[ClassificationPerformanceMetrics()])
report.run(current_data=merged_test_data, reference_data=merged_train_data, column_mapping=iris_column_mapping)
report

## Probabilistic Model Performance Dashboard short (verbose_level=0)

In [33]:
iris_prob_classification_dashboard = Dashboard(tabs=[ProbClassificationPerformanceTab(verbose_level=0)])
iris_prob_classification_dashboard.calculate(merged_train_data, merged_test_data, 
                                             column_mapping = iris_column_mapping)
iris_prob_classification_dashboard.show()

In [34]:
# iris_prob_classification_dashboard.save('iris_prob_classification_dashboard.html')

## Model Performance Profile

In [35]:
iris_prob_classification_profile = Profile(sections=[ProbClassificationPerformanceProfileSection()])

In [36]:
iris_prob_classification_profile.calculate(merged_train_data, merged_test_data, 
                                           column_mapping = iris_column_mapping)

In [37]:
iris_prob_classification_profile.json() 

'{"probabilistic_classification_performance": {"name": "probabilistic_classification_performance", "datetime": "2022-01-03 02:09:21.477109", "data": {"utility_columns": {"date": null, "id": null, "target": "target", "prediction": ["setosa", "versicolor", "virginica"]}, "cat_feature_names": [], "num_feature_names": ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"], "target_names": null, "metrics": {"reference": {"accuracy": 0.9821428571428571, "precision": 0.9844961240310077, "recall": 0.9803921568627452, "f1": 0.981962481962482, "roc_auc": 0.9990392020420109, "log_loss": 0.12458988316075034, "metrics_matrix": {"setosa": {"precision": 1.0, "recall": 1.0, "f1-score": 1.0, "support": 37}, "versicolor": {"precision": 1.0, "recall": 0.9411764705882353, "f1-score": 0.9696969696969697, "support": 34}, "virginica": {"precision": 0.9534883720930233, "recall": 1.0, "f1-score": 0.9761904761904763, "support": 41}, "accuracy": 0.9821428571428571, "macro avg": {"prec